In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
pd.options.display.max_colwidth = 400
from IPython.display import FileLink, FileLinks

In [ ]:
train = pd.read_csv('/kaggle/input/web-traffic-time-series-forecasting/train_2.csv.zip')
keys = pd.read_csv('/kaggle/input/web-traffic-time-series-forecasting/key_2.csv.zip')
sub = pd.read_csv('/kaggle/input/web-traffic-time-series-forecasting/sample_submission_2.csv.zip')

In [ ]:
train.head()

In [ ]:
keys.head()

In [ ]:
sub.head()

### What is the date range to predict?

In [ ]:
def split_page_col(page):
    tokens = page.split('_')
    article_name = ''.join(tokens[:-3])
    org = tokens[-3]
    access = tokens[-2]
    crawler = tokens[-1]
    return (article_name, org, access, crawler)

def split_page_col_wdate(page):
    tokens = page.split('_')
    article_name = ''.join(tokens[:-4])
    org = tokens[-4]
    access = tokens[-3]
    crawler = tokens[-2]
    date = tokens[-1]
    return (article_name, org, access, crawler,date)

In [ ]:
keys['date'] = keys['Page'].apply(lambda x: x.split('_')[-1])
keys['Page'] = keys['Page'].apply(lambda x: '_'.join(x.split('_')[:-1]))
keys['date'] = pd.to_datetime(keys['date'], format='%Y-%m-%d')

In [ ]:
keys.tail()

In [ ]:
sub = sub.merge(keys, on='Id', how='left')

In [ ]:
sub['date'] = pd.to_datetime(sub['date'], format='%Y-%m-%d')

In [ ]:
print(sub['date'].min(), sub['date'].max())
print(sub['date'].max() - sub['date'].min())

In [ ]:
print(sub['Page'].nunique())
print(train['Page'].nunique())

#### Predict for the next 61 days for all pages

## Baselines Predictions

- Baseline 1 - Predict the same value as the previous year (LB - 62.65)
- Baseline 2 - Predict the median of the Page (LB - 47.11656)
- Baseline 3 - Predict the median in this date range the previous year (LB - 53.60537)
- Baseline 4 - Predict the median of that week from the previous year //TODO

- Baseline 5 - Predict the median of the past 60 days 

In [ ]:
train.shape

In [ ]:
train.iloc[:, 755:803]

In [ ]:
print(sub.shape)
print(train.shape)

In [ ]:
prev_year_data_cols = pd.date_range('2016-09-13', '2016-11-13')
train_flat = pd.melt(train.loc[:, ['Page'] + list(prev_year_data_cols.date.astype(str))], id_vars='Page', var_name='date')
train_flat['date'] = pd.to_datetime(train_flat['date'], format='%Y-%m-%d')

In [ ]:
train_flat.head()

In [ ]:
train_flat['prediction_date'] = train_flat['date'] + pd.DateOffset(years=1)
sub = sub[['Page', 'date', 'Id']].merge(train_flat[['Page', 'prediction_date', 'value']], left_on=('Page', 'date'), right_on=('Page', 'prediction_date'))
sub['value'] = sub['value'].fillna(0)

In [ ]:
sub[['Id', 'value']].rename(columns={'value': 'visits'}).to_csv('all_submission.csv', index=False)
FileLink('all_submission.csv')

## Baseline 2 median of the Page

In [ ]:
page_median = train.iloc[:, 1:].median(axis=1, skipna=True)

In [ ]:
page_median = pd.DataFrame({'Page': train['Page'], 'median': page_median})

In [ ]:
page_median.head()

In [ ]:
sub_median = sub.merge(page_median, on='Page')[['Id', 'median']]

In [ ]:
sub_median.isnull().mean()

In [ ]:
sub_median

In [ ]:
sub_median.rename(columns={'median': 'visits'}).to_csv('submission.csv', index=False)
FileLink('submission.csv')

## Baseline 3 - median of the page in the prev year 

In [ ]:
prev_year_data_cols = pd.date_range('2016-09-13', '2016-11-13')
prev_year_median = train.loc[:, list(prev_year_data_cols.date.astype(str))].median(axis=1, skipna=True)


In [ ]:
prev_year_median = pd.DataFrame({'Page': train['Page'], 'visits': prev_year_median})

In [ ]:
sub_prev_year_median = sub.merge(prev_year_median, on='Page')[['Id', 'visits']]

In [ ]:
sub_prev_year_median.isnull().mean()

In [ ]:
sub_prev_year_median['visits'] = sub_prev_year_median['visits'].fillna(sub_prev_year_median['visits'].median())

In [ ]:
# sub_prev_year_median.to_csv('submission_prev_year.csv',index=False)
# FileLink('submission_prev_year.csv')

## Baseline 5 - Median of past 60 days

In [ ]:
median_60 = train.iloc[:, -60:].median(axis=1, skipna=True)
median_60 = pd.DataFrame({'Page': train['Page'], 'visits': median_60})
sub_median_60 = sub.merge(median_60, on='Page')[['Id', 'visits']]

In [ ]:
sub_median_60.isnull().mean()

In [ ]:
sub_median_60['visits'] = sub_median_60['visits'].fillna(0)

In [ ]:
# sub_prev_year_median.to_csv('sub_median_60.csv',index=False)
# FileLink('sub_median_60.csv')